In [ ]:
%pip install pydantic
%pip install langchain
%pip install langchain_community


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 21.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53934 sha256=892dcaae9b859edcd9590c2efe46a256505330783432012e441a8b8725888d88
  Stored in directory: /Users/timleibold/Library/Caches/pip/wheels/f3/ba/dd/43ed5f165600f41deddeb1e382c56ffc1067c09ec5bd705f39
Successfully built docx
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pydantic import BaseModel, Field

class PsychotherapieAntrag(BaseModel):
    anrede: str = Field(..., description="Anrede, z.B. 'Sehr geehrte Frau Dr. Müller,'")
    relevante_soziodemographische_daten: str = Field(..., description="1. Relevante soziodemographische Daten: aktuell ausgeübter Beruf, Familienstand, Zahl der Kinder, Lebenssituation, Gecshwister und ähnliches")
    symptomatik_psychischer_befund: str = Field(..., description="2. Symptomatik und psychischer Befund: Von Patient geschuilderte Symptomatik, Angaben zu Schwere und verlauf; Auffälligkeiten bei Kontaktaufnahme, Interaktion, Erscheinungsbild; Psychischer Befund; Krankheitsverständnis des Patienten; oder ähnliches")
    somatischer_befund: str = Field(..., description="3. Somatischer Befund / Konsiliarbericht: Somatische Befunde, gegebenenfalls aktuelle psychopharmakologische Medikation, Vorbehandlungen")
    lebensgeschichtliche_angaben: str = Field(
        ..., description="4. Behandlungsrelevante Angaben zur Lebensgeschichte / Psychosomatik / Systemisches Erklärungsmodell: "
    )
    diagnose_zeitpunkt: str = Field(..., description="5. Diagnose zum Zeitpunkt der Antragstellung")
    behandlungsplan_prognose: str = Field(..., description="6. Behandlungsplan und Prognose: Beschreibung der konkreten, mit dem Patient reflektierten Therapieziele, individueller krankheitsvezogener Behandlungsplan;begründung des Settings; Kooperation mit anderen Berufsgruppen; und Ähnliches")
    
    signatur: str = Field(..., description="Unterschrift / Signaturblock")

In [11]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

# 1) Erstelle das JSON‐Gerüst als String (hier indent=2 nur für Lesbarkeit)
json_schema = PsychotherapieAntrag.schema_json(indent=2, sort_keys=False)

system = SystemMessagePromptTemplate.from_template(
    """Du bist ein Assistent, der aus dem gegebenen Transkript einen vollständigen Psychotherapie-Antrag (§7-Struktur) in sauberem JSON erstellt. 
    Dabei beachtest du die folgenden Punkte:
    1. Der Antrag ist auf Deutsch.
    2. Der Antrag wird in förmlicher Sprache verfasst.
    3. Es dürfen keine Informationen hinzugefügt werden, die nicht im Transkript enthalten sind, außer es handelt sich um Klarstellungen."""
)

human = HumanMessagePromptTemplate.from_template("""
Hier ist das Transkript der Audio-Memo (auf Deutsch). Bitte fülle die Felder des folgenden JSON-Schemas **WORTGENAU**, etwa:
  
Transkript:
\"\"\"{transkript}\"\"\"

JSON-Schema:
```json
{json_schema}
Antworte nur mit gültigem JSON, das exakt dem Schema entspricht""")

prompt = ChatPromptTemplate.from_messages([system, human])

/var/folders/fj/jtvwfjnn7dj6_2nctj66wvhr0000gn/T/ipykernel_27917/2460279849.py:4: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json_schema = PsychotherapieAntrag.schema_json(indent=2, sort_keys=False)


In [17]:
transkript_text = """
Hallo, hier spricht Dr. Sabine Meier, Psychologische Psychotherapeutin, Praxis am Lindenweg in München. Ich nehme dieses Memo auf, um die wichtigsten Eckdaten und Befunde meiner Patientin Martina Muster festzuhalten.

Martina Muster ist 38 Jahre alt, verheiratet, Mutter einer sechsjährigen Tochter und einer dreijährigen Tochter. Sie arbeitet Teilzeit als Grafikdesignerin in einem Werbeunternehmen. Ihre Familie lebt in einem Einfamilienhaus am Stadtrand, sie hat keine Geschwister. Martina berichtet, dass sie tagsüber häufig Erschöpfung und Konzentrationsschwierigkeiten verspürt, was sich negativ auf ihre Kreativität im Beruf auswirkt.

In Bezug auf die Symptomatik beschreibt sie seit etwa sechs Monaten eine anhaltende Niedergestimmtheit und Anhedonie. Sie könne nichts mehr richtig genießen, weder Spaziergänge noch Treffen mit Freundinnen. Gelegentlich klagt sie über Einschlaf­störungen und frühes Erwachen, verbunden mit gedanklichem Grübeln — vor allem über berufliche Fehler oder Konflikte in ihrer Ehe. Ihr Affekt sei oft verflacht, mit gelegentlichen Stimmungseinbrüchen und Gefühlen von Wertlosigkeit. Beim Erstgespräch wirkte sie leicht verzögert in der Motorik, mit gedrückter Mimik und zurückhaltender Stimme. Der psychische Befund deutet auf eine leichte bis mittelgradige depressive Episode hin, mit somatischer Komponente in Form von innerer Unruhe und Muskelverspannungen.

Somatisch ist Martina ansonsten unauffällig. In der Vorgeschichte existiert eine Schilddrüsen­unterfunktion, die medikamentös gut eingestellt ist. Aktuell nimmt sie kein Psychopharmakon ein. Ein kardiologisches Konsil im letzten Quartal ergab einen normalen Blutdruck und unauffällige EKG-Befunde. Vorbehandlung: Im Jahr 2023 eine vierwöchige Physiotherapie wegen Schulterneuralgie, ansonsten keine relevante somatische Vorerkrankung. Es liegen keine Hinweise auf psychosomatische Bauchschmerzen oder vegetative Dystonie vor.

Zur Lebensgeschichte: Martina wuchs als Einzelkind in einer stabilen Familie auf, hatte jedoch früh hohen Leistungsdruck in der Schule. Ihre Mutter litt selbst phasenweise unter Angststörungen, was zu einer emotional distanzierten Eltern-Kind-Beziehung führte. Martina entwickelte in der Adoleszenz eine hohe Perfektionismustendenz und erlernte adaptive Coping-Strategien wie exzessives Arbeiten und Rückzug. Seit der Geburt ihrer zweiten Tochter vor drei Jahren fühle sie sich jedoch zunehmend überfordert und isoliert. Die partnerschaftliche Unterstützung sei begrenzt, ihr Mann arbeite im Schichtdienst. Sie beschreibt ein systemisches Erklärungsmodell, in dem familiäre Rollenmuster und elterlicher Anteilnahme Einfluss auf ihre Selbstwahrnehmung und ihr Stressregulationsverhalten haben.

Diagnose zum Zeitpunkt der Antragstellung: Leichte depressive Episode (ICD-10 F32.0) mit typischen Symptomen von Anhedonie, Verminderung des Selbstwertgefühls und leichtem Schlafdefizit.

Für den Behandlungsplan wurden gemeinsam mit Martina folgende Therapieziele festgelegt:  
- Verbesserung der Schlafhygiene und Reduktion des Grübelns durch strukturierte Tagebuchführung und achtsamkeitsbasierte Übungen.  
- Steigerung der positiven Aktivitäten (Behavioral Activation), z. B. tägliche Spaziergänge und regelmäßige Treffen mit Freundinnen.  
- Bearbeitung von Perfektionismus und dysfunktionalen Kognitionen mithilfe kognitiver Umstrukturierung.  
- Förderung von Ressourcen und Stärkung des Selbstwertgefühls durch klientenzentrierte Interventionen.  

Wir empfehlen ein initiales Setting mit wöchentlich einer 50-minütigen Sitzung für mindestens zwölf Sitzungen (mittelfristige Zielsetzung), um Stabilisierung und Symptomreduktion zu erreichen. Eine interdisziplinäre Kooperation mit dem Hausarzt und bei Bedarf einer Physiotherapeutin zur Entspannung der Muskelverspannungen ist geplant.

Das war’s für das erste Memo. Ende des Transkripts.

"""

In [28]:
transkript_text = """Hallo, ähm, hier spricht Dr. Sabine Meier aus der Praxis am Lindenweg in München. Martina Muster, 38 Jahre alt, verheiratet, lebt mit ihrem Mann und zwei Töchtern (sechs und drei Jahre) in einem Einfamilienhaus am Stadtrand und arbeitet Teilzeit als Grafikdesignerin in einem Werbeunternehmen. Sie hat keine Geschwister.

Hmm, sie berichtet seit etwa sechs Monaten von anhaltender Niedergeschlagenheit und Anhedonie – Spaziergänge oder Treffen mit Freundinnen machen ihr kaum noch Freude. Tagsüber fühlt sie sich oft erschöpft und hat Konzentrationsprobleme, was ihre Kreativität im Job erheblich einschränkt. Einschlaf­störungen und frühes Erwachen sind häufig, verbunden mit Grübeln über berufliche Fehler und Konflikte in der Ehe. Affekt abgeflacht, gelegentliche Stimmungseinbrüche und Wertlosigkeitsgefühle. Beim Erstgespräch wirkte sie motorisch leicht verzögert, mit gedrückter Mimik und zurückhaltender Stimme.

Somatisch ist sie ansonsten unauffällig; die bekannte Schilddrüsenunterfunktion ist gut eingestellt, und aktuell nimmt sie kein Psychopharmakon. Ein kardiologisches Konsil im letzten Quartal ergab normalen Blutdruck und unauffälliges EKG. 2023 hatte sie eine vierwöchige Physiotherapie wegen einer Schulterneuralgie, ansonsten keinerlei relevante Vorerkrankungen – keine psychosomatischen Bauchschmerzen oder vegetative Dystonie.

Martina wuchs als Einzelkind in einer Familie auf, in der ihre Mutter phasenweise unter Angststörungen litt, was zu einer eher distanzierten Beziehung führte. Früh entstand hoher Leistungsdruck in der Schule, sie entwickelte Perfektionismustendenzen und adaptive Coping-Strategien: exzessives Arbeiten oder Rückzug. Seit der Geburt ihrer zweiten Tochter fühlt sie sich zunehmend überfordert und isoliert, der Schichtdienst ihres Mannes erschwert die partnerschaftliche Unterstützung.

Diagnostisch ergibt sich eine leichte depressive Episode (ICD-10 F32.0) mit typischen Symptomen von Anhedonie, vermindertem Selbstwert und leichtem Schlafdefizit. Wir haben gemeinsam Therapieziele definiert: Verbesserung der Schlafhygiene durch strukturierte Tagebuchführung und achtsamkeitsbasierte Übungen, Steigerung positiver Aktivitäten (tägliche Spaziergänge, regelmäßige Treffen mit Freundinnen), kognitive Umstrukturierung zur Bearbeitung von Perfektionismus und dysfunktionalen Gedanken sowie Förderung von Ressourcen und Stärkung des Selbstwertgefühls durch klientenzentrierte Interventionen.

Für das initiale Setting empfehle ich wöchentliche 50-minütige Sitzungen über mindestens zwölf Einheiten, um Stabilisierung und Symptomreduktion zu erreichen. Eine interdisziplinäre Zusammenarbeit mit dem Hausarzt und gegebenenfalls einer Physiotherapeutin zur Entspannung der Muskelverspannungen ist vorgesehen. Ende des Memos.
"""

In [13]:

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
import os

#  Instantiate LLM (hier GPT-4, deutschsprachig, temperatur niedrig für Konsistenz)
llm = ChatOpenAI(model="gpt-4", temperature=0.0, api_key=os.getenv("OPENAI_API_KEY"))

chain = LLMChain(llm=llm, prompt=prompt)

# Transkript z.B. aus Deinem Speech-to-Text-Modul
transkript_text = """Frau Schmidt, 45 Jahre, ..."""

result = chain.run(transkript=transkript_text, json_schema=json_schema)

# `result` ist jetzt ein JSON-String; parse ihn in Dein Pydantic-Model:
antrag = PsychotherapieAntrag.parse_raw(result)
print(antrag.dict())

/var/folders/fj/jtvwfjnn7dj6_2nctj66wvhr0000gn/T/ipykernel_27917/3780790631.py:16: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  antrag = PsychotherapieAntrag.parse_raw(result)


ValidationError: 1 validation error for PsychotherapieAntrag
__root__
  Expecting value: line 1 column 1 (char 0) [type=value_error.jsondecode, input_value='Transkript:\n"""\nSehr g...n, Dr. Müller"\n}\n```', input_type=str]

In [ ]:
from pydantic import BaseModel, Field
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
import os

def LLMCall(transkript_text: str) -> dict:
    # 1) Dein Pydantic-Schema
    class PsychotherapieAntrag(BaseModel):
        anrede: str = Field(..., description="Anrede, z.B. 'Sehr geehrte Frau Dr. Müller,'")
        relevante_soziodemographische_daten: str = Field(
            ..., description="1. Relevante soziodemographische Daten: aktuell ausgeübter Beruf, Familienstand, Zahl der Kinder, Lebenssituation, Geschwister und ähnliches"
        )
        symptomatik_psychischer_befund: str = Field(
            ..., description="2. Symptomatik und psychischer Befund: Von Patientin geschilderte Symptomatik, Angaben zu Schwere und Verlauf; Auffälligkeiten bei Kontaktaufnahme, Interaktion, Erscheinungsbild; psychischer Befund; Krankheitsverständnis"
        )
        somatischer_befund: str = Field(
            ..., description="3. Somatischer Befund / Konsiliarbericht: Somatische Befunde, aktuelle psychopharmakologische Medikation, Vorbehandlungen"
        )
        lebensgeschichtliche_angaben: str = Field(
            ..., description="4. Behandlungsrelevante Angaben zur Lebensgeschichte / Psychosomatik / Systemisches Erklärungsmodell"
        )
        diagnose_zeitpunkt: str = Field(
            ..., description="5. Diagnose zum Zeitpunkt der Antragstellung"
        )
        behandlungsplan_prognose: str = Field(
            ..., description="6. Behandlungsplan und Prognose: Therapieziele, Behandlungsplan, Setting-Begründung, Kooperation"
        )
        signatur: str = Field(
            ..., description="Unterschrift / Signaturblock"
        )


    # 2) Erzeuge den Parser aus Deinem Model
    parser = PydanticOutputParser(pydantic_object=PsychotherapieAntrag)
    format_instructions = parser.get_format_instructions()

    # 3) Baue Dein Prompt
    system = SystemMessagePromptTemplate.from_template(
        "Du bist ein Assistent, der aus dem gegebenen Transkript einen vollständigen Psychotherapie-Antrag "
        "(6 Abschnitte plus Anrede und Signatur) **ausschließlich als JSON** gemäß dem Pydantic-Schema erstellt. "
        "Sprache: Deutsch. Förmlich, psychologische Fachsprache, keine zusätzlichen Informationen."
    )

    human = HumanMessagePromptTemplate.from_template("""
    Hier das Transkript der Audio-Memo (auf Deutsch):
    \"\"\"{transkript}\"\"\"

    {format_instructions}
    """)

    prompt = ChatPromptTemplate.from_messages([system, human])

    # 4) LLM und Chain initialisieren
    llm = ChatOpenAI(
        model="gpt-4",
        temperature=0.0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        output_parser=parser
    )

    # 5) Transkript (in einer anderen Zelle definiert)
    # z.B.:
    # transkript_text = """Frau Meier, 38 Jahre, ..."""

    # 6) Aufruf und direktes Parsen
    antrag_obj: PsychotherapieAntrag = chain.predict_and_parse(
        transkript=transkript_text,
        format_instructions=format_instructions
    )
    data = antrag_obj.model_dump()
    return data
# 7) Nutze Dein Objekt
# Option A: model_dump_json (Pydantic v2)
# gibt valid JSON mit Einrückung zurück, aber escapet non-ASCII-Zeichen
print(antrag_obj.model_dump_json(indent=2))

/opt/anaconda3/envs/referral/lib/python3.12/site-packages/langchain/chains/llm.py:342: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{
  "anrede": "Sehr geehrte Damen und Herren,",
  "relevante_soziodemographische_daten": "Martina Muster, 38 Jahre alt, verheiratet, lebt mit ihrem Mann und zwei Töchtern (sechs und drei Jahre) in einem Einfamilienhaus am Stadtrand und arbeitet Teilzeit als Grafikdesignerin in einem Werbeunternehmen. Sie hat keine Geschwister.",
  "symptomatik_psychischer_befund": "Sie berichtet seit etwa sechs Monaten von anhaltender Niedergeschlagenheit und Anhedonie – Spaziergänge oder Treffen mit Freundinnen machen ihr kaum noch Freude. Tagsüber fühlt sie sich oft erschöpft und hat Konzentrationsprobleme, was ihre Kreativität im Job erheblich einschränkt. Einschlaf­störungen und frühes Erwachen sind häufig, verbunden mit Grübeln über berufliche Fehler und Konflikte in der Ehe. Affekt abgeflacht, gelegentliche Stimmungseinbrüche und Wertlosigkeitsgefühle. Beim Erstgespräch wirkte sie motorisch leicht verzögert, mit gedrückter Mimik und zurückhaltender Stimme.",
  "somatischer_befund": "Somatisch ist

In [20]:
import pprint as pp
pp.pprint(antrag_obj.model_dump_json(indent=2))

('{\n'
 '  "anrede": "Sehr geehrte Damen und Herren,",\n'
 '  "relevante_soziodemographische_daten": "Martina Muster ist 38 Jahre alt, '
 'verheiratet, Mutter einer sechsjährigen Tochter und einer dreijährigen '
 'Tochter. Sie arbeitet Teilzeit als Grafikdesignerin in einem '
 'Werbeunternehmen. Ihre Familie lebt in einem Einfamilienhaus am Stadtrand, '
 'sie hat keine Geschwister.",\n'
 '  "symptomatik_psychischer_befund": "Martina berichtet, dass sie tagsüber '
 'häufig Erschöpfung und Konzentrationsschwierigkeiten verspürt, was sich '
 'negativ auf ihre Kreativität im Beruf auswirkt. In Bezug auf die Symptomatik '
 'beschreibt sie seit etwa sechs Monaten eine anhaltende Niedergestimmtheit '
 'und Anhedonie. Sie könne nichts mehr richtig genießen, weder Spaziergänge '
 'noch Treffen mit Freundinnen. Gelegentlich klagt sie über '
 'Einschlaf\xadstörungen und frühes Erwachen, verbunden mit gedanklichem '
 'Grübeln — vor allem über berufliche Fehler oder Konflikte in ihrer Ehe. Ihr '
 '

In [ ]:
import json
from docx import Document

# --- after your chain.predict_and_parse call you have: 
# antrag_obj: PsychotherapieAntrag

# Convert Pydantic model to plain dict
  # v2 Pydantic; if v1 use .dict()

# Map field names to their human-readable titles
titles = {
    'relevante_soziodemographische_daten': 
        'Relevante soziodemographische Daten',
    'symptomatik_psychischer_befund': 
        'Symptomatik und psychischer Befund',
    'somatischer_befund': 
        'Somatischer Befund / Konsiliarbericht',
    'lebensgeschichtliche_angaben':
        'Behandlungsrelevante Angaben zur Lebensgeschichte / Psychosomatik / Systemisches Erklärungsmodell',
    'diagnose_zeitpunkt':
        'Diagnose zum Zeitpunkt der Antragstellung',
    'behandlungsplan_prognose':
        'Behandlungsplan und Prognose',
}

# Create the document
doc = Document()

# 1) Anrede
doc.add_paragraph(data['anrede'])

# 2) Numbered sections 1–6
for key in titles:
    # Add a numbered heading
    p_heading = doc.add_paragraph(style='List Number')
    p_heading.add_run(f"{titles[key]}")
    # Add the user’s text for that section
    doc.add_paragraph(data[key])

# 3) Signatur
doc.add_paragraph(data['signatur'])

# Save
doc_path = "Psychotherapie_Antrag.docx"
doc.save(doc_path)
print(f"Dokument gespeichert als: {doc_path}")

Dokument gespeichert als: Psychotherapie_Antrag.docx
